In [1]:
import csv
import json
import pathlib


In [2]:
places_file = pathlib.Path("../data/places.json")
raw_data = places_file.read_text()
places = json.loads(raw_data)

In [3]:
NORTH_JERSEY_COUNTIES = (
    "Bergen County",
    "Essex County",
    "Hudson County",
    "Morris County",
    "Passaic County",
    "Sussex County",
    "Warren County",
)

CENTRAL_JERSEY_COUNTIES = (
    "Hunterdon County",
    "Mercer County",
    "Middlesex County",
    "Somerset County",
    "Union County",
)

SOUTH_JERSEY_COUNTIES = (
    "Burlington County",
    "Camden County",
    "Cumberland County",
    "Gloucester County",
    "Salem County",
)


JERSEY_SHORE_COUNTIES = (
    "Atlantic County",
    "Cape May County",
    "Monmouth County",
    "Ocean County",
)

In [4]:
def county_to_region(county: str) -> str:
    if county in NORTH_JERSEY_COUNTIES:
        return "North Jersey"
    elif county in CENTRAL_JERSEY_COUNTIES:
        return "Central Jersey"
    elif county in SOUTH_JERSEY_COUNTIES:
        return "South Jersey"
    elif county in JERSEY_SHORE_COUNTIES:
        return "Jersey Shore"
    else:
        raise RuntimeError(f"could not map county {county} to region.")


In [5]:
def get_addr_component(addr_components, location_type: str):
    return next(filter(lambda p: location_type in p.get("types", []), addr_components))

In [7]:
flat_places = []
for place in places.values():
    county = get_addr_component(place["addressComponents"], "administrative_area_level_2")["longText"]
    record = {
        "display_name": place["displayName"]["text"],
        "primary_type": place["primaryType"],
        "county": county,
        "region": county_to_region(county),
        "website": place.get("websiteUri", "")
        
    }
    flat_places.append(record)

In [10]:
flat_places[:5]

[{'display_name': "Kinchley's Tavern",
  'primary_type': 'pizza_restaurant',
  'county': 'Bergen County',
  'region': 'North Jersey',
  'website': 'https://www.kinchleyspizza.com/'},
 {'display_name': 'Mama Rosa Pizza and Pasta',
  'primary_type': 'pizza_restaurant',
  'county': 'Bergen County',
  'region': 'North Jersey',
  'website': 'https://www.mamarosapasta.com/'},
 {'display_name': 'Aquila Pizza Al Forno',
  'primary_type': 'pizza_restaurant',
  'county': 'Passaic County',
  'region': 'North Jersey',
  'website': 'http://aquilaforno.com/'},
 {'display_name': 'Novino Ristorante',
  'primary_type': 'italian_restaurant',
  'county': 'Bergen County',
  'region': 'North Jersey',
  'website': 'http://www.novinonj.com/'},
 {'display_name': 'The Pizza Terminal & Restaurant NJ',
  'primary_type': 'pizza_restaurant',
  'county': 'Essex County',
  'region': 'North Jersey',
  'website': 'https://www.thepizzaterminalnj.com/'}]

In [9]:
output_file = pathlib.Path("../data/places.csv")
with output_file.open('w') as f:
    writer = csv.DictWriter(f, fieldnames=list(flat_places[0].keys()))
    writer.writeheader()
    writer.writerows(flat_places)
